In [1]:
import pandas as pd
import numpy as np
import pyprojroot.here as here

In [2]:
data1 = pd.read_csv(here("data/processed/cleaned_swissotel-the-stamford.csv"))
data2 = pd.read_csv(here("data/processed/cleaned_mbs_total.csv"))
data3 = pd.read_csv(here("data/processed/cleaned_pan-pacific.csv"))
data4 = pd.read_csv(here("data/processed/cleaned_parkroyal-collection-marina-bay.csv"))
data5 = pd.read_csv(here("data/processed/cleaned_fullerton.csv"))
data = pd.concat([data1, data2, data3, data4, data5], ignore_index = True)
print(len(data1))
print(len(data2))
print(len(data3))
print(len(data4))
print(len(data5))
len(data)

5058
10523
7430
6237
6374


35622

In [3]:
data.head()

,Unnamed: 0,traveller_username,review_title,review_text,travel_type,traveller_country_origin,traveller_total_contributions,traveller_total_helpful_contributions,rating,valid_rating,label,cleaned_review,combined_review,date,covid,year,stem_review,lem_review
0,0,Ernest L,Excellent stay with fantastic scenery,This is our 4th stay. I still remember that I ...,Trip type: Travelled with family,NaN,1.0,NaN,5.0,True,Positive,excellent stay fantastic scenery th stay still...,Excellent stay with fantastic scenery This is ...,2023-09-01,PostCovid,2023,excel stay fantast sceneri th stay still remem...,excellent stay fantastic scenery th stay still...
1,1,lovemylife999,Pampercation,64th floor Crest Suite Harbour view. Spacious ...,Trip type: Travelled as a couple,"Singapore, Singapore",91.0,2.0,NaN,False,NaN,pampercation th floor crest suite harbour view...,Pampercation 64th floor Crest Suite Harbour vi...,2023-05-01,PostCovid,2023,pamperc th floor crest suit harbour view spaci...,pampercation th floor crest suite harbour view...
2,2,JOSE JOAQUIN ORTIZ GARCIA,Excellent location and facilities as a confere...,"The hotel is located near the marina, so going...",Trip type: Travelled on business,"Chia, Colombia",32.0,NaN,5.0,True,Positive,excellent location facilities conference venue...,Excellent location and facilities as a confere...,2023-09-01,PostCovid,2023,excel locat facil confer venu hotel locat near...,excellent location facility conference venue h...
3,3,Changboo C,Great experience while staying the Swissotel,Was the great staying. I have watched the whol...,Trip type: Travelled on business,NaN,1.0,NaN,5.0,True,Positive,great experience staying swissotel great stayi...,Great experience while staying the Swissotel W...,2023-09-01,PostCovid,2023,great experi stay swissotel great stay watch w...,great experience stay swissotel great staying ...
4,4,Traveller,"A decent stay, something can improve!",A decent stay with Swissotel The Stamford sinc...,Trip type: Travelled on business,NaN,1.0,NaN,3.0,True,Neutral,decent stay something improve decent stay swis...,"A decent stay, something can improve! A decent...",2023-09-01,PostCovid,2023,decent stay someth improv decent stay swissote...,decent stay something improve decent stay swis...


In [4]:
!pip install pyabsa --quiet --root-user-action=ignore

In [5]:
from pyabsa import ATEPCCheckpointManager
from pyabsa import available_checkpoints
import re 
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings("ignore")

/bin/sh: nvidia-smi: command not found
/Users/ammarbagharib/miniforge3/envs/bt4222_hotels/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No CUDA GPU found in your device
[2023-11-09 00:22:41] (2.3.4) PyABSA(2.3.4): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.x versions, e.g., pip install pyabsa<2.0 -U




Try to downgrade transformers<=4.29.0.






/Users/ammarbagharib/miniforge3/envs/bt4222_hotels/lib/python3.10/multiprocessing/pool.py:265: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",


In [6]:
text = list(data['lem_review'][30:35])
text

['another amazing trip stay night th august amazing stay find staff friendly rhea staff club lounge look we well feel spoil hotel close mrt shopping centre thankyou hotel staff make stay memorable anne michael',
 'disappointing hotel provide great service last night staff bother turn bed etc do every night leave next morning disappointing hotel standard',
 'room view lovely hotel great location easy access shopping transport etc upgrade room view boy view staff super helpful professional would stay singapore without doubt breakfast variety excellent delicious',
 'twin bed room fit family love hotel twin bed big enough family really plus point family like we get welcome fruit room check nice chocolate cake big one everyday view room amazing love come back visit singapore',
 'spectacular view spectacular view helpful staff spacious room easily accessible property however toilet bidet nonetheless family wonderful stay love marina bay view amazing truly beautiful']

In [7]:
checkpoint_map = available_checkpoints()

[2023-11-09 00:22:59] (2.3.4) Please specify the task code, e.g. from pyabsa import TaskCodeOption


In [8]:
aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(checkpoint='english',
                                   auto_device=True  # False means load model on CPU
                                   )

[2023-11-09 00:23:01] (2.3.4) Downloading checkpoint:english 
[2023-11-09 00:23:01] (2.3.4) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets


Find zipped checkpoint: ./checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43.zip, unzipping


Done.
[2023-11-09 00:23:17] (2.3.4) If the auto-downloading failed, please download it via browser: https://huggingface.co/spaces/yangheng/PyABSA/resolve/main/checkpoints/English/ATEPC/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43.zip 
[2023-11-09 00:23:17] (2.3.4) Load aspect extractor from checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43
[2023-11-09 00:23:17] (2.3.4) config: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43/fast_lcf_atepc.config
[2023-11-09 00:23:17] (2.3.4) state_dict: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43/fast_lcf_atepc.state_dict
[2023-11-09 00:23:17] (2.3.4) model: None
[2023-11-09 00:23:17] (2.3.4) tokenizer: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43/fast_lcf_atepc.tokenizer
[2023-11-09 00:23:17] (2.3.4) Set Model Device: c

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
atepc_result = aspect_extractor.extract_aspect(inference_source=text,  #
                          pred_sentiment=True,  # Predict the sentiment of extracted aspect terms
                          )

[2023-11-09 00:23:34] (2.3.4) The results of aspect term extraction have been saved in /Users/ammarbagharib/git/sentiment_analysis_bt4222/codes/aspect_modelling/Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json
[2023-11-09 00:23:34] (2.3.4) Example 0: another amazing trip stay night th august amazing stay find <staff:Positive Confidence:0.9973> friendly rhea <staff club:Positive Confidence:0.9648> lounge look we well feel spoil hotel close mrt shopping centre thankyou hotel staff make stay memorable anne michael
[2023-11-09 00:23:34] (2.3.4) Example 1: disappointing hotel provide great <service:Positive Confidence:0.9585> last night <staff:Negative Confidence:0.8588> bother turn bed etc do every night leave next morning disappointing hotel standard
[2023-11-09 00:23:34] (2.3.4) Example 2: room view lovely hotel great <location:Positive Confidence:0.9994> easy access shopping transport etc upgrade <room view:Positive Confidence:0.999> boy view <staff:Positive

In [11]:
atepc_result[0:5]

[{'sentence': 'another amazing trip stay night th august amazing stay find staff friendly rhea staff club lounge look we well feel spoil hotel close mrt shopping centre thankyou hotel staff make stay memorable anne michael',
  'IOB': ['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-ASP',
   'O',
   'O',
   'B-ASP',
   'I-ASP',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O'],
  'tokens': ['another',
   'amazing',
   'trip',
   'stay',
   'night',
   'th',
   'august',
   'amazing',
   'stay',
   'find',
   'staff',
   'friendly',
   'rhea',
   'staff',
   'club',
   'lounge',
   'look',
   'we',
   'well',
   'feel',
   'spoil',
   'hotel',
   'close',
   'mrt',
   'shopping',
   'centre',
   'thankyou',
   'hotel',
   'staff',
   'make',
   'stay',
   'memorable',
   'anne',
   'michael'],
  'aspect': ['staff', 'staff club'],
  'position': [[10], [1